In [36]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from linearmodels import IV2SLS, IVGMM, IVGMMCUE
from scipy.stats import f
from IPython.display import Latex

In [37]:
date_bound = '2012-05-01'

# 1 

a.	將value_factor_all.xlsx/ mom_factor_all.xlsx 中的TW_equity 、JP_equity 、US_equity和Global_equity_indice.xlsx中對應國家的return做迴歸（類似CAPM）無風險利率用Global_government_bonds.xlsx中應國家的return）


In [38]:
val = pd.read_excel('value_factor_all.xlsx')
val = val[val.Date>=date_bound].reset_index(drop = True)
val.Name = 'val'
mom = pd.read_excel('mom_factor_all.xlsx')
mom = mom[mom.Date>=date_bound].reset_index(drop = True)
mom.Name = 'mom'
couns = ['TW', 'JP', 'US']

equi = pd.ExcelFile('Global_equity_indice.xlsx')
rf_excel = pd.ExcelFile('Global_government_bonds.xlsx')
res_1a = pd.DataFrame(columns=['country', 'type', 'alpha', 't_value'])

for type in [val, mom]:
    for coun in couns:
        x_col = type.filter(like=coun, axis=1).columns
        xs = type[x_col]

        ys = pd.read_excel(equi, coun)
        ys =  ys[ys.Date>=date_bound].reset_index(drop = True)
        rf = pd.read_excel(rf_excel, coun)
        rf =  rf[rf.Date>=date_bound].reset_index(drop = True)
        for_model = pd.DataFrame({f'{coun}_{type.Name}' : xs.iloc[:, 0],  # 這邊不確定要不要扣rf，反正有先把rf抓出來了
                                  f'{coun}_indice' : ys['return']})

                
        x = for_model.iloc[:, 0]
        x = sm.add_constant(x)
        y = for_model.iloc[:, 1]

        model = sm.OLS(y, x)

        results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1}) # Newey-West std
        alpha = results.params['const']
        const_t_stat = results.tvalues['const']
        res_1a.loc[len(res_1a)] = [coun, type.Name, alpha, const_t_stat]
        # print(results.summary())

In [39]:
res_1a

,country,type,alpha,t_value
0,TW,val,0.004743,1.501164
1,JP,val,0.007160,1.832033
2,US,val,0.011288,1.532972
3,TW,mom,0.007036,2.151198
4,JP,mom,0.010304,2.556450
5,US,mom,0.015194,2.198686


b.	將value_factor_all.xlsx/ mom_factor_all.xlsx 中的Global_equity_indice和MSCI_world.xlsx中的mkt_return做迴歸
無風險利率用Global_government_bonds.xlsx中美國的return


In [40]:
val = pd.read_excel('value_factor_all.xlsx')
val = val[val.Date>=date_bound].reset_index(drop = True)[['Global_equity_indice']]
val.Name = 'val'
mom = pd.read_excel('mom_factor_all.xlsx')
mom = mom[mom.Date>=date_bound].reset_index(drop = True)[['Global_equity_indice']]
mom.Name = 'mom'

msciworld = pd.read_excel('MSCI_world.xlsx')
msciworld = msciworld[msciworld.Date>=date_bound].reset_index(drop = True)[['mkt_return']]
msciworld = msciworld.rename(columns = {'mkt_return' : 'msci_mkt_return'})

rf_excel = pd.ExcelFile('Global_government_bonds.xlsx')
res_1b = pd.DataFrame(columns=['type', 'alpha', 't_value'])

for type in [val, mom]:
    
    xs = type

    ys = msciworld

    rf = pd.read_excel(rf_excel, 'US')
    rf =  rf[rf.Date>=date_bound].reset_index(drop = True)
    for_model = pd.DataFrame({f'global_equity_{type.Name}' : xs.iloc[:, 0],  # 這邊不確定要不要扣rf，反正有先把rf抓出來了
                                'MSCI' : ys.iloc[:, 0]})

            
    x = for_model.iloc[:, 0]
    x = sm.add_constant(x)
    y = for_model.iloc[:, 1]

    model = sm.OLS(y, x)

    results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1}) # Newey-West std
    alpha = results.params['const']
    const_t_stat = results.tvalues['const']
    res_1b.loc[len(res_1b)] = [type.Name, alpha, const_t_stat]
    # print(results.summary())

In [41]:
res_1b

,type,alpha,t_value
0,val,0.006688,2.243308
1,mom,0.007178,2.254541


# 2

a.	將每個countryname_port_mom/value.xlsx中的不同Portfolio（一個檔案有三個，所以總共會跑九次）和MSCI_world.xlsx中的mkt_return、value_factor_all.xlsx中的value_everywhere、mom_factor_all.xlsx中的mom_everywhere做迴歸無風險利率用Global_government_bonds.xlsx中應國家的return）
這部分的回歸公式對應於論文中的(5)

b.	要做GRS test !! (好像是錯的)


$R^p_{it} - r^p_{f, t} = \alpha^p_i+\beta^p_iMKT_t+v^p_iVAL^{EVERYWHERE}_t+m^p_iMOM^{everywhere}_t+\epsilon^p_{it}$

In [42]:
# x
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats import f

msciworld = pd.read_excel('MSCI_world.xlsx')
msciworld = msciworld[msciworld.Date>=date_bound].reset_index(drop = True)[['mkt_return']]
val = pd.read_excel('value_factor_all.xlsx')
val = val[val.Date>=date_bound].reset_index(drop = True)[['value_everywhere']]
mom = pd.read_excel('mom_factor_all.xlsx')
mom = mom[mom.Date>=date_bound].reset_index(drop = True)[['mom_everywhere']]
rf_excel = pd.ExcelFile('Global_government_bonds.xlsx')
res_2 = pd.DataFrame(columns=['country', 'type', 'rank', 'alpha'])

# List of portfolio files
files = ['JP_port_mom.xlsx', 'JP_port_value.xlsx', 'TW_port_mom.xlsx', 'TW_port_value.xlsx', 'US_port_mom.xlsx', 'US_port_value.xlsx']

# Initialize variables for alphas and residuals
alphas = []
residuals = []
T = 0  # Total number of observations

# Iterate over files and portfolios
for file in files:
    xs = pd.read_excel(file)
    coun = file.split('_')[0]
    type = file.split('_')[2].split('.')[0]
    xs = xs[xs.Date >= date_bound].reset_index(drop=True)
    rf = pd.read_excel(rf_excel, coun)
    rf = rf[rf.Date >= date_bound].reset_index(drop=True)['return']

    for i in range(1, 5):
        # Define the return portfolio
        # rp_rf = xs.iloc[:, i] - rf  
        # rank = f'P{i}'
        # Model data
        for_model = pd.DataFrame({'mkt': msciworld['mkt_return'], 'val': val['value_everywhere'], 'mom': mom['mom_everywhere']})
        X = sm.add_constant(for_model.iloc[:, :3])

        if i == 4:
            y = xs.iloc[:, 3] - xs.iloc[:, 1]
        else:
            y = xs.iloc[:, i] - rf
        # y = for_model['rp-rf']

        # Fit the model
        model = sm.OLS(y, X)
        results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
        
        # Collect alphas and residuals
        alphas.append(results.params['const'])
        residuals.append(results.resid)

        # Update total number of observations
        T += results.nobs
        rank = 'P3-P1' if i == 4 else f'P{i}'
        res_2.loc[len(res_2)] = [coun, type, rank, results.params['const']]

# Average number of observations per portfolio
T_avg = T / len(alphas)

# Calculate the covariance matrix of residuals and its inverse
residual_cov = np.cov(np.column_stack(residuals), rowvar=False)
inv_residual_cov = np.linalg.inv(residual_cov)

# Number of portfolios and factors
N = len(alphas)
K = 3

# Calculate GRS statistic and p-value
alphas = np.array(alphas)
grs_stat = (T_avg - N - K) / N * (T_avg - 1) / T_avg * alphas @ inv_residual_cov @ alphas
grs_p_value = 1 - f.cdf(grs_stat, N, T_avg - N - K)

grs_stat, grs_p_value

(20.95872268960882, 1.1102230246251565e-16)

In [43]:
xs.iloc[:, 1]

0     -0.047258
1      0.026652
2      0.065242
3      0.019709
4      0.031841
         ...   
135    0.017513
136   -0.058566
137   -0.005704
138    0.088365
139    0.050401
Name: Top Third Portfolio, Length: 140, dtype: float64

In [44]:
res_2

,country,type,rank,alpha
0,JP,mom,P1,-0.033745
1,JP,mom,P2,0.007443
2,JP,mom,P3,0.031094
3,JP,mom,P3-P1,0.064839
4,JP,value,P1,-0.020435
5,JP,value,P2,0.002634
6,JP,value,P3,0.014774
7,JP,value,P3-P1,0.035209
8,TW,mom,P1,-0.005952
9,TW,mom,P2,0.020704


# 3

a.	將countryname_port_mom/value.xlsx中的不同Portfolio用FF 3 factors和5 factors再跑一次，一樣要做GRS test !!
還沒做

$R^p_{it} - r^p_{f, t} = \alpha^p_i+\beta^p_iMKT_t+v^p_iVAL^{EVERYWHERE}_t+m^p_iMOM^{everywhere}_t+\epsilon^p_{it}$

## FF3

In [45]:
# x
ff3 = pd.read_csv('F-F_Research_Data_3_Factors.csv')
ff3 = ff3[pd.to_datetime(ff3['date']) >= pd.to_datetime(date_bound)].reset_index(drop = True)
ff5 = pd.read_csv('F-F_Research_Data_5_Factors_2x3.csv')
ff5 = ff5[pd.to_datetime(ff5['date']) >= pd.to_datetime(date_bound)].reset_index(drop = True)

rf_excel = pd.ExcelFile('Global_government_bonds.xlsx')
res_3_ff3 = pd.DataFrame(columns=['country', 'type', 'rank', 'alpha', 't_value'])
files = ['JP_port_mom.xlsx', 'JP_port_value.xlsx', 'TW_port_mom.xlsx', 'TW_port_value.xlsx', 'US_port_mom.xlsx', 'US_port_value.xlsx']

# Initialize variables for alphas and residuals
alphas = []
residuals = []
T = 0  # Total number of observations

for file in files:
    xs = pd.read_excel(file)
    file = file.replace('.xlsx', '')
    coun = file.split('_')[0]
    type = file.split('_')[2]
    xs = xs[xs.Date>=date_bound]
    xs = xs[xs.Date<=pd.to_datetime('2022-12-31')].reset_index(drop = True)
    rf = pd.read_excel(rf_excel, coun)
    rf =  rf[rf.Date>=date_bound]
    rf = rf[rf.Date<=pd.to_datetime('2022-12-31')].reset_index(drop = True)['return']
    for i in range(1, 5):
        if i == 4:
            y = xs.iloc[:, 3] - xs.iloc[:, 1]
        else:
            y = xs.iloc[:, i] - rf

        X = sm.add_constant(ff3.iloc[:, 1:4])
        # y = for_model.iloc[:, -1]
        model = sm.OLS(y, X)
        rank = f'P{i}'
        results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1}) # Newey-West std
        alpha = results.params['const']
        const_t_stat = results.tvalues['const']

        # grs test
        # Collect alphas and residuals
        alphas.append(results.params['const'])
        residuals.append(results.resid)

        # Update total number of observations
        T += results.nobs
        
        rank = 'P3-P1' if i == 4 else f'P{i}'
        res_3_ff3.loc[len(res_3_ff3)] = [coun, type, rank, alpha, const_t_stat]

# Average number of observations per portfolio
T_avg = T / len(alphas)

# Calculate the covariance matrix of residuals and its inverse
residual_cov = np.cov(np.column_stack(residuals), rowvar=False)
inv_residual_cov = np.linalg.inv(residual_cov)

# Number of portfolios and factors
N = len(alphas)
K = 3

# Calculate GRS statistic and p-value
alphas = np.array(alphas)
grs_stat = (T_avg - N - K) / N * (T_avg - 1) / T_avg * alphas @ inv_residual_cov @ alphas
grs_p_value = 1 - f.cdf(grs_stat, N, T_avg - N - K)

grs_stat, grs_p_value

(3.486563207442662, 5.898169929796637e-06)

In [46]:
res_3_ff3

,country,type,rank,alpha,t_value
0,JP,mom,P1,-0.004026,-0.644559
1,JP,mom,P2,0.007080,1.334095
2,JP,mom,P3,0.006612,1.192441
3,JP,mom,P3-P1,0.010638,1.331279
4,JP,value,P1,-0.005084,-0.974018
5,JP,value,P2,-0.001701,-0.340686
6,JP,value,P3,0.012020,1.891367
7,JP,value,P3-P1,0.017104,2.425329
8,TW,mom,P1,-0.003598,-0.895702
9,TW,mom,P2,-0.001335,-0.406513


## FF5

In [47]:
# x
ff5 = pd.read_csv('F-F_Research_Data_5_Factors_2x3.csv')
ff5 = ff5[pd.to_datetime(ff5['date']) >= pd.to_datetime(date_bound)].reset_index(drop = True)

rf_excel = pd.ExcelFile('Global_government_bonds.xlsx')
res_3_ff5 = pd.DataFrame(columns=['country', 'type', 'rank', 'alpha', 't_value'])
files = ['JP_port_mom.xlsx', 'JP_port_value.xlsx', 'TW_port_mom.xlsx', 'TW_port_value.xlsx', 'US_port_mom.xlsx', 'US_port_value.xlsx']

# Initialize variables for alphas and residuals
alphas = []
residuals = []
T = 0  # Total number of observations

for file in files:
    xs = pd.read_excel(file)
    file = file.replace('.xlsx', '')
    coun = file.split('_')[0]
    type = file.split('_')[2]
    xs = xs[xs.Date>=date_bound]
    xs = xs[xs.Date<=pd.to_datetime('2022-12-31')].reset_index(drop = True)
    rf = pd.read_excel(rf_excel, coun)
    rf =  rf[rf.Date>=date_bound]
    rf = rf[rf.Date<=pd.to_datetime('2022-12-31')].reset_index(drop = True)['return']
    for i in range(1, 5):
        if i == 4:
            y = xs.iloc[:, 3] - xs.iloc[:, 1]
        else:
            y = xs.iloc[:, i] - rf

        X = sm.add_constant(ff5.iloc[:, 1:6])
        # y = for_model.iloc[:, -1]
        model = sm.OLS(y, X)
        rank = f'P{i}'
        results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1}) # Newey-West std
        alpha = results.params['const']
        const_t_stat = results.tvalues['const']

        # grs test
        # Collect alphas and residuals
        alphas.append(results.params['const'])
        residuals.append(results.resid)

        # Update total number of observations
        T += results.nobs
        
        rank = 'P3-P1' if i == 4 else f'P{i}'
        res_3_ff5.loc[len(res_3_ff5)] = [coun, type, rank, alpha, const_t_stat]

# Average number of observations per portfolio
T_avg = T / len(alphas)

# Calculate the covariance matrix of residuals and its inverse
residual_cov = np.cov(np.column_stack(residuals), rowvar=False)
inv_residual_cov = np.linalg.inv(residual_cov)

# Number of portfolios and factors
N = len(alphas)
K = 5

# Calculate GRS statistic and p-value
alphas = np.array(alphas)
grs_stat = (T_avg - N - K) / N * (T_avg - 1) / T_avg * alphas @ inv_residual_cov @ alphas
grs_p_value = 1 - f.cdf(grs_stat, N, T_avg - N - K)

grs_stat, grs_p_value

(0.5307308974122037, 0.9613897193853319)

In [48]:
res_3_ff5

,country,type,rank,alpha,t_value
0,JP,mom,P1,-0.002005,-0.331198
1,JP,mom,P2,0.009386,1.773249
2,JP,mom,P3,0.006692,1.147264
3,JP,mom,P3-P1,0.008697,1.079855
4,JP,value,P1,-0.003625,-0.703374
5,JP,value,P2,-0.000433,-0.087006
6,JP,value,P3,0.014629,2.275972
7,JP,value,P3-P1,0.018254,2.652999
8,TW,mom,P1,-0.004537,-1.241547
9,TW,mom,P2,-0.001930,-0.590827
